In [ ]:
/* ============================================================================
   Setup: Database and Schema
   ============================================================================ */
USE DATABASE CSV_FINAL_ASSIGNMENT;
CREATE SCHEMA IF NOT EXISTS SILVER;

/* ============================================================================
   Helper UDF for string normalization (unchanged)
   ============================================================================ */
CREATE OR REPLACE FUNCTION SILVER.NVL_STR(V VARCHAR)
RETURNS VARCHAR
AS $$
  COALESCE(NULLIF(TRIM(V), ''), 'Unknown')
$$;

/* ============================================================================
   Target table definition (unchanged)
   ============================================================================ */
CREATE OR REPLACE TABLE CSV_FINAL_ASSIGNMENT.SILVER.SILVER_PATIENT (
  patient_id             VARCHAR(64)  NOT NULL,
  first_name             VARCHAR(128),
  last_name              VARCHAR(128),
  middle                 VARCHAR(128),
  prefix                 VARCHAR(32),
  suffix                 VARCHAR(32),
  maiden                 VARCHAR(128),
  birth_date             TIMESTAMP_NTZ,
  death_date             TIMESTAMP_NTZ,
  gender                 VARCHAR(32),
  marital_status         VARCHAR(32),
  ethnicity              VARCHAR(64),
  ssn                    VARCHAR(16),
  drivers_license        VARCHAR(64),
  passport               VARCHAR(64),
  birthplace             VARCHAR(256),
  address                VARCHAR(256),
  city                   VARCHAR(128),
  state                  VARCHAR(64),
  zip                    VARCHAR(32),
  county                 VARCHAR(128),
  fips                   VARCHAR(16),
  lat                    NUMBER(9,6),
  lon                    NUMBER(9,6),
  healthcare_expenses    NUMBER(18,2),
  healthcare_coverage    NUMBER(9,4),
  income                 NUMBER(18,2),
  source_system          VARCHAR(64)  NOT NULL,
  record_source_id       VARCHAR(256),
  created_at             TIMESTAMP_NTZ DEFAULT CURRENT_TIMESTAMP(),
  updated_at             TIMESTAMP_NTZ DEFAULT CURRENT_TIMESTAMP()
);

/* ============================================================================
   Initial load (unchanged logic, one-time bulk insert)
   ============================================================================ */
INSERT INTO SILVER.SILVER_PATIENT (
  PATIENT_ID, FIRST_NAME, LAST_NAME, MIDDLE, PREFIX, SUFFIX, MAIDEN,
  BIRTH_DATE, DEATH_DATE, GENDER, MARITAL_STATUS, ETHNICITY, SSN,
  DRIVERS_LICENSE, PASSPORT, BIRTHPLACE, ADDRESS, CITY, STATE, ZIP, COUNTY, FIPS,
  LAT, LON, HEALTHCARE_EXPENSES, HEALTHCARE_COVERAGE, INCOME,
  SOURCE_SYSTEM, RECORD_SOURCE_ID, CREATED_AT, UPDATED_AT
)
WITH csv_pat AS (
  SELECT
    SILVER.NVL_STR(CAST(Id AS VARCHAR(64)))                    AS patient_id,
    SILVER.NVL_STR(CAST(FIRST      AS VARCHAR(128)))           AS first_name,
    SILVER.NVL_STR(CAST(LAST       AS VARCHAR(128)))           AS last_name,
    SILVER.NVL_STR(NULL)                                        AS middle,
    SILVER.NVL_STR(CAST(PREFIX     AS VARCHAR(32)))            AS prefix,
    SILVER.NVL_STR(CAST(SUFFIX     AS VARCHAR(32)))            AS suffix,
    SILVER.NVL_STR(CAST(MAIDEN     AS VARCHAR(128)))           AS maiden,

    TRY_TO_TIMESTAMP_NTZ(BIRTHDATE)                            AS birth_date,
    TRY_TO_TIMESTAMP_NTZ(DEATHDATE)                            AS death_date,

    SILVER.NVL_STR(CAST(GENDER     AS VARCHAR(32)))            AS gender,
    SILVER.NVL_STR(CAST(MARITAL    AS VARCHAR(32)))            AS marital_status,
    SILVER.NVL_STR(CAST(ETHNICITY  AS VARCHAR(64)))            AS ethnicity,
    SILVER.NVL_STR(CAST(SSN        AS VARCHAR(16)))            AS ssn,
    SILVER.NVL_STR(CAST(DRIVERS    AS VARCHAR(64)))            AS drivers_license,
    SILVER.NVL_STR(CAST(PASSPORT   AS VARCHAR(64)))            AS passport,
    SILVER.NVL_STR(CAST(BIRTHPLACE AS VARCHAR(256)))           AS birthplace,
    SILVER.NVL_STR(CAST(ADDRESS    AS VARCHAR(256)))           AS address,
    SILVER.NVL_STR(CAST(CITY       AS VARCHAR(128)))           AS city,
    SILVER.NVL_STR(CAST(STATE      AS VARCHAR(64)))            AS state,
    SILVER.NVL_STR(CAST(ZIP        AS VARCHAR(32)))            AS zip,
    SILVER.NVL_STR(CAST(COUNTY     AS VARCHAR(128)))           AS county,
    SILVER.NVL_STR(CAST(FIPS       AS VARCHAR(16)))            AS fips,

    TRY_TO_NUMBER(LAT)                                         AS lat,
    TRY_TO_NUMBER(LON)                                         AS lon,
    TRY_TO_NUMBER(HEALTHCARE_EXPENSES)                         AS healthcare_expenses,
    CASE
      WHEN TRY_TO_NUMBER(HEALTHCARE_COVERAGE) > 1
        THEN TRY_TO_NUMBER(HEALTHCARE_COVERAGE) / 100
      ELSE TRY_TO_NUMBER(HEALTHCARE_COVERAGE)
    END                                                        AS healthcare_coverage,
    TRY_TO_NUMBER(INCOME)                                      AS income,

    'CSV'                                                      AS source_system,
    SILVER.NVL_STR(CONCAT('CSV:patients.csv|', CAST(Id AS VARCHAR(64)))) AS record_source_id,

    CURRENT_TIMESTAMP()                                        AS created_at,
    CURRENT_TIMESTAMP()                                        AS updated_at
  FROM CSV_FINAL_ASSIGNMENT.CSV.PATIENTS
),
hl7_pid AS (
  SELECT
    SILVER.NVL_STR(CAST(PATIENT_ID_LIST AS VARCHAR(64)))       AS patient_id,
    SILVER.NVL_STR(CAST(first_name AS VARCHAR(128)))           AS first_name,
    SILVER.NVL_STR(CAST(last_name  AS VARCHAR(128)))           AS last_name,
    SILVER.NVL_STR(NULL)                                        AS middle,
    SILVER.NVL_STR(NULL)                                        AS prefix,
    SILVER.NVL_STR(NULL)                                        AS suffix,
    SILVER.NVL_STR(NULL)                                        AS maiden,

    TO_TIMESTAMP_NTZ(DOB_DATE)                                  AS birth_date,
    NULL                                                        AS death_date,

    SILVER.NVL_STR(CAST(SEX AS VARCHAR(32)))                    AS gender,
    SILVER.NVL_STR(NULL)                                        AS marital_status,
    SILVER.NVL_STR(NULL)                                        AS ethnicity,
    SILVER.NVL_STR(NULL)                                        AS ssn,
    SILVER.NVL_STR(NULL)                                        AS drivers_license,
    SILVER.NVL_STR(NULL)                                        AS passport,
    SILVER.NVL_STR(NULL)                                        AS birthplace,

    SILVER.NVL_STR(CONCAT(STREET_1, STREET_2))                  AS address,
    SILVER.NVL_STR(CAST(CITY        AS VARCHAR(128)))           AS city,
    SILVER.NVL_STR(CAST(STATE       AS VARCHAR(64)))            AS state,
    SILVER.NVL_STR(CAST(POSTAL_CODE AS VARCHAR(32)))            AS zip,
    SILVER.NVL_STR(NULL)                                        AS county,
    SILVER.NVL_STR(NULL)                                        AS fips,

    NULL                                                        AS lat,
    NULL                                                        AS lon,
    NULL                                                        AS healthcare_expenses,
    NULL                                                        AS healthcare_coverage,
    NULL                                                        AS income,

    'HL7'                                                       AS source_system,
    SILVER.NVL_STR(CONCAT('HL7:PID|', CAST(MESSAGE_ID AS VARCHAR(256)))) AS record_source_id,

    CURRENT_TIMESTAMP()                                         AS created_at,
    CURRENT_TIMESTAMP()                                         AS updated_at
  FROM HL7_FINAL_ASSIGNMENT.HL7.HL7_BRONZE_PID
),
ccda_pat AS (
  SELECT
    SILVER.NVL_STR(CAST(PATIENT_ID_EXTENSION AS VARCHAR(64)))     AS patient_id,
    SILVER.NVL_STR(CAST(GIVEN_NAME            AS VARCHAR(128)))   AS first_name,
    SILVER.NVL_STR(CAST(FAMILY_NAME           AS VARCHAR(128)))   AS last_name,
    SILVER.NVL_STR(NULL)                                           AS middle,
    SILVER.NVL_STR(NULL)                                           AS prefix,
    SILVER.NVL_STR(NULL)                                           AS suffix,
    SILVER.NVL_STR(NULL)                                           AS maiden,

    TO_TIMESTAMP_NTZ(DATE_OF_BIRTH)                                AS birth_date,
    NULL                                                           AS death_date,

    SILVER.NVL_STR(CAST(GENDER_CODE         AS VARCHAR(32)))       AS gender,
    SILVER.NVL_STR(NULL)                                           AS marital_status,
    SILVER.NVL_STR(CAST(ETHNICITY_CODE      AS VARCHAR(64)))       AS ethnicity,

    SILVER.NVL_STR(NULL)                                           AS ssn,
    SILVER.NVL_STR(NULL)                                           AS drivers_license,
    SILVER.NVL_STR(NULL)                                           AS passport,
    SILVER.NVL_STR(NULL)                                           AS birthplace,

    SILVER.NVL_STR(CAST(STREET_ADDRESS_LINE AS VARCHAR(256)))      AS address,
    SILVER.NVL_STR(CAST(CITY                AS VARCHAR(128)))      AS city,
    SILVER.NVL_STR(CAST(STATE               AS VARCHAR(64)))       AS state,
    SILVER.NVL_STR(CAST(POSTAL_CODE         AS VARCHAR(32)))       AS zip,
    SILVER.NVL_STR(NULL)                                           AS county,
    SILVER.NVL_STR(NULL)                                           AS fips,

    NULL                                                           AS lat,
    NULL                                                           AS lon,
    NULL                                                           AS healthcare_expenses,
    NULL                                                           AS healthcare_coverage,
    NULL                                                           AS income,

    'CCDA'                                                         AS source_system,
    SILVER.NVL_STR(CONCAT('CCDA:V_PATIENT_DEMOGRAPHICS|',
                          CAST(PATIENT_ID_EXTENSION AS VARCHAR(256)))) AS record_source_id,

    CURRENT_TIMESTAMP()                                            AS created_at,
    CURRENT_TIMESTAMP()                                            AS updated_at
  FROM CCDA_FINAL_ASSIGNMENT.CCDA.V_PATIENT_DEMOGRAPHICS
)
SELECT * FROM csv_pat
UNION ALL
SELECT * FROM hl7_pid
UNION ALL
SELECT * FROM ccda_pat
;

/* ============================================================================
   Manual Refresh (no streams, no scheduled task) — run whenever needed
   Duplicates are NOT deleted; we simply pick one row per key to satisfy MERGE.
   ============================================================================ */
MERGE INTO SILVER.SILVER_PATIENT T
USING (
  /* ---------------------- CSV (collapse duplicates in USING only) ---------------------- */
  WITH csv_full AS (
    SELECT
      CAST(Id AS VARCHAR(64))                                   AS raw_patient_id,
      SILVER.NVL_STR(CAST(Id AS VARCHAR(64)))                   AS patient_id,
      SILVER.NVL_STR(CAST(FIRST      AS VARCHAR(128)))          AS first_name,
      SILVER.NVL_STR(CAST(LAST       AS VARCHAR(128)))          AS last_name,
      SILVER.NVL_STR(NULL)                                       AS middle,
      SILVER.NVL_STR(CAST(PREFIX     AS VARCHAR(32)))           AS prefix,
      SILVER.NVL_STR(CAST(SUFFIX     AS VARCHAR(32)))           AS suffix,
      SILVER.NVL_STR(CAST(MAIDEN     AS VARCHAR(128)))          AS maiden,
      TRY_TO_TIMESTAMP_NTZ(BIRTHDATE)                           AS birth_date,
      TRY_TO_TIMESTAMP_NTZ(DEATHDATE)                           AS death_date,
      SILVER.NVL_STR(CAST(GENDER     AS VARCHAR(32)))           AS gender,
      SILVER.NVL_STR(CAST(MARITAL    AS VARCHAR(32)))           AS marital_status,
      SILVER.NVL_STR(CAST(ETHNICITY  AS VARCHAR(64)))           AS ethnicity,
      SILVER.NVL_STR(CAST(SSN        AS VARCHAR(16)))           AS ssn,
      SILVER.NVL_STR(CAST(DRIVERS    AS VARCHAR(64)))           AS drivers_license,
      SILVER.NVL_STR(CAST(PASSPORT   AS VARCHAR(64)))           AS passport,
      SILVER.NVL_STR(CAST(BIRTHPLACE AS VARCHAR(256)))          AS birthplace,
      SILVER.NVL_STR(CAST(ADDRESS    AS VARCHAR(256)))          AS address,
      SILVER.NVL_STR(CAST(CITY       AS VARCHAR(128)))          AS city,
      SILVER.NVL_STR(CAST(STATE      AS VARCHAR(64)))           AS state,
      SILVER.NVL_STR(CAST(ZIP        AS VARCHAR(32)))           AS zip,
      SILVER.NVL_STR(CAST(COUNTY     AS VARCHAR(128)))          AS county,
      SILVER.NVL_STR(CAST(FIPS       AS VARCHAR(16)))           AS fips,
      TRY_TO_NUMBER(LAT)                                        AS lat,
      TRY_TO_NUMBER(LON)                                        AS lon,
      TRY_TO_NUMBER(HEALTHCARE_EXPENSES)                        AS healthcare_expenses,
      CASE
        WHEN TRY_TO_NUMBER(HEALTHCARE_COVERAGE) > 1
          THEN TRY_TO_NUMBER(HEALTHCARE_COVERAGE) / 100
        ELSE TRY_TO_NUMBER(HEALTHCARE_COVERAGE)
      END                                                       AS healthcare_coverage,
      TRY_TO_NUMBER(INCOME)                                     AS income,
      'CSV'                                                     AS source_system,
      SILVER.NVL_STR(CONCAT('CSV:patients.csv|', CAST(Id AS VARCHAR(64)))) AS record_source_id,
      CURRENT_TIMESTAMP()                                       AS created_at,
      CURRENT_TIMESTAMP()                                       AS updated_at
    FROM CSV_FINAL_ASSIGNMENT.CSV.PATIENTS
    QUALIFY ROW_NUMBER() OVER (
      PARTITION BY raw_patient_id
      /* choose a stable representative; adjust ordering if you have a better timestamp */
      ORDER BY record_source_id DESC
    ) = 1
  ),

  /* ---------------------- HL7 (collapse duplicates in USING only) ---------------------- */
  hl7_full AS (
    SELECT
      CAST(PATIENT_ID_LIST AS VARCHAR(64))                      AS raw_patient_id,
      SILVER.NVL_STR(CAST(PATIENT_ID_LIST AS VARCHAR(64)))      AS patient_id,
      SILVER.NVL_STR(CAST(first_name AS VARCHAR(128)))          AS first_name,
      SILVER.NVL_STR(CAST(last_name  AS VARCHAR(128)))          AS last_name,
      SILVER.NVL_STR(NULL)                                       AS middle,
      SILVER.NVL_STR(NULL)                                       AS prefix,
      SILVER.NVL_STR(NULL)                                       AS suffix,
      SILVER.NVL_STR(NULL)                                       AS maiden,
      TO_TIMESTAMP_NTZ(DOB_DATE)                                 AS birth_date,
      NULL                                                       AS death_date,
      SILVER.NVL_STR(CAST(SEX AS VARCHAR(32)))                  AS gender,
      SILVER.NVL_STR(NULL)                                       AS marital_status,
      SILVER.NVL_STR(NULL)                                       AS ethnicity,
      SILVER.NVL_STR(NULL)                                       AS ssn,
      SILVER.NVL_STR(NULL)                                       AS drivers_license,
      SILVER.NVL_STR(NULL)                                       AS passport,
      SILVER.NVL_STR(NULL)                                       AS birthplace,
      SILVER.NVL_STR(CONCAT(STREET_1, STREET_2))                AS address,
      SILVER.NVL_STR(CAST(CITY        AS VARCHAR(128)))         AS city,
      SILVER.NVL_STR(CAST(STATE       AS VARCHAR(64)))          AS state,
      SILVER.NVL_STR(CAST(POSTAL_CODE AS VARCHAR(32)))          AS zip,
      SILVER.NVL_STR(NULL)                                       AS county,
      SILVER.NVL_STR(NULL)                                       AS fips,
      NULL                                                      AS lat,
      NULL                                                      AS lon,
      NULL                                                      AS healthcare_expenses,
      NULL                                                      AS healthcare_coverage,
      NULL                                                      AS income,
      'HL7'                                                     AS source_system,
      SILVER.NVL_STR(CONCAT('HL7:PID|', CAST(MESSAGE_ID AS VARCHAR(256)))) AS record_source_id,
      CURRENT_TIMESTAMP()                                       AS created_at,
      CURRENT_TIMESTAMP()                                       AS updated_at
    FROM HL7_FINAL_ASSIGNMENT.HL7.HL7_BRONZE_PID
    QUALIFY ROW_NUMBER() OVER (
      PARTITION BY raw_patient_id
      /* prefer a stable choice; MESSAGE_ID often increases */
      ORDER BY record_source_id DESC
    ) = 1
  ),

  /* ---------------------- CCDA (collapse duplicates in USING only) --------------------- */
  ccda_full AS (
    SELECT
      CAST(PATIENT_ID_EXTENSION AS VARCHAR(64))                 AS raw_patient_id,
      SILVER.NVL_STR(CAST(PATIENT_ID_EXTENSION AS VARCHAR(64))) AS patient_id,
      SILVER.NVL_STR(CAST(GIVEN_NAME            AS VARCHAR(128))) AS first_name,
      SILVER.NVL_STR(CAST(FAMILY_NAME           AS VARCHAR(128))) AS last_name,
      SILVER.NVL_STR(NULL)                                       AS middle,
      SILVER.NVL_STR(NULL)                                       AS prefix,
      SILVER.NVL_STR(NULL)                                       AS suffix,
      SILVER.NVL_STR(NULL)                                       AS maiden,
      TO_TIMESTAMP_NTZ(DATE_OF_BIRTH)                            AS birth_date,
      NULL                                                       AS death_date,
      SILVER.NVL_STR(CAST(GENDER_CODE         AS VARCHAR(32)))   AS gender,
      SILVER.NVL_STR(NULL)                                       AS marital_status,
      SILVER.NVL_STR(CAST(ETHNICITY_CODE      AS VARCHAR(64)))   AS ethnicity,
      SILVER.NVL_STR(NULL)                                       AS ssn,
      SILVER.NVL_STR(NULL)                                       AS drivers_license,
      SILVER.NVL_STR(NULL)                                       AS passport,
      SILVER.NVL_STR(NULL)                                       AS birthplace,
      SILVER.NVL_STR(CAST(STREET_ADDRESS_LINE  AS VARCHAR(256))) AS address,
      SILVER.NVL_STR(CAST(CITY                 AS VARCHAR(128))) AS city,
      SILVER.NVL_STR(CAST(STATE                AS VARCHAR(64)))  AS state,
      SILVER.NVL_STR(CAST(POSTAL_CODE          AS VARCHAR(32)))  AS zip,
      SILVER.NVL_STR(NULL)                                       AS county,
      SILVER.NVL_STR(NULL)                                       AS fips,
      NULL                                                       AS lat,
      NULL                                                       AS lon,
      NULL                                                       AS healthcare_expenses,
      NULL                                                       AS healthcare_coverage,
      NULL                                                       AS income,
      'CCDA'                                                     AS source_system,
      SILVER.NVL_STR(CONCAT('CCDA:V_PATIENT_DEMOGRAPHICS|',
                            CAST(PATIENT_ID_EXTENSION AS VARCHAR(256)))) AS record_source_id,
      CURRENT_TIMESTAMP()                                        AS created_at,
      CURRENT_TIMESTAMP()                                        AS updated_at
    FROM CCDA_FINAL_ASSIGNMENT.CCDA.V_PATIENT_DEMOGRAPHICS
    QUALIFY ROW_NUMBER() OVER (
      PARTITION BY raw_patient_id
      ORDER BY record_source_id DESC
    ) = 1
  )

  SELECT * FROM csv_full
  UNION ALL
  SELECT * FROM hl7_full
  UNION ALL
  SELECT * FROM ccda_full
) S
ON T.PATIENT_ID = NULLIF(TRIM(S.raw_patient_id), '')

/* Upserts — unchanged field-level logic */
WHEN MATCHED THEN UPDATE SET
  T.FIRST_NAME          = COALESCE(NULLIF(TRIM(S.FIRST_NAME), ''),          SILVER.NVL_STR(T.FIRST_NAME)),
  T.LAST_NAME           = COALESCE(NULLIF(TRIM(S.LAST_NAME),  ''),          SILVER.NVL_STR(T.LAST_NAME)),
  T.MIDDLE              = COALESCE(NULLIF(TRIM(S.MIDDLE),     ''),          SILVER.NVL_STR(T.MIDDLE)),
  T.PREFIX              = COALESCE(NULLIF(TRIM(S.PREFIX),     ''),          SILVER.NVL_STR(T.PREFIX)),
  T.SUFFIX              = COALESCE(NULLIF(TRIM(S.SUFFIX),     ''),          SILVER.NVL_STR(T.SUFFIX)),
  T.MAIDEN              = COALESCE(NULLIF(TRIM(S.MAIDEN),     ''),          SILVER.NVL_STR(T.MAIDEN)),
  T.BIRTH_DATE          = COALESCE(S.BIRTH_DATE, T.BIRTH_DATE),
  T.DEATH_DATE          = COALESCE(S.DEATH_DATE, T.DEATH_DATE),
  T.GENDER              = COALESCE(NULLIF(TRIM(S.GENDER),     ''),          SILVER.NVL_STR(T.GENDER)),
  T.MARITAL_STATUS      = COALESCE(NULLIF(TRIM(S.MARITAL_STATUS), ''),      SILVER.NVL_STR(T.MARITAL_STATUS)),
  T.ETHNICITY           = COALESCE(NULLIF(TRIM(S.ETHNICITY),  ''),          SILVER.NVL_STR(T.ETHNICITY)),
  T.SSN                 = COALESCE(NULLIF(TRIM(S.SSN),        ''),          SILVER.NVL_STR(T.SSN)),
  T.DRIVERS_LICENSE     = COALESCE(NULLIF(TRIM(S.DRIVERS_LICENSE), ''),     SILVER.NVL_STR(T.DRIVERS_LICENSE)),
  T.PASSPORT            = COALESCE(NULLIF(TRIM(S.PASSPORT),   ''),          SILVER.NVL_STR(T.PASSPORT)),
  T.BIRTHPLACE          = COALESCE(NULLIF(TRIM(S.BIRTHPLACE), ''),          SILVER.NVL_STR(T.BIRTHPLACE)),
  T.ADDRESS             = COALESCE(NULLIF(TRIM(S.ADDRESS),    ''),          SILVER.NVL_STR(T.ADDRESS)),
  T.CITY                = COALESCE(NULLIF(TRIM(S.CITY),       ''),          SILVER.NVL_STR(T.CITY)),
  T.STATE               = COALESCE(NULLIF(TRIM(S.STATE),      ''),          SILVER.NVL_STR(T.STATE)),
  T.ZIP                 = COALESCE(NULLIF(TRIM(S.ZIP),        ''),          SILVER.NVL_STR(T.ZIP)),
  T.COUNTY              = COALESCE(NULLIF(TRIM(S.COUNTY),     ''),          SILVER.NVL_STR(T.COUNTY)),
  T.FIPS                = COALESCE(NULLIF(TRIM(S.FIPS),       ''),          SILVER.NVL_STR(T.FIPS)),
  T.LAT                 = COALESCE(S.LAT, T.LAT),
  T.LON                 = COALESCE(S.LON, T.LON),
  T.HEALTHCARE_EXPENSES = COALESCE(S.HEALTHCARE_EXPENSES, T.HEALTHCARE_EXPENSES),
  T.HEALTHCARE_COVERAGE = COALESCE(S.HEALTHCARE_COVERAGE, T.HEALTHCARE_COVERAGE),
  T.INCOME              = COALESCE(S.INCOME, T.INCOME),
  T.SOURCE_SYSTEM       = COALESCE(NULLIF(TRIM(S.SOURCE_SYSTEM), ''),       SILVER.NVL_STR(T.SOURCE_SYSTEM)),
  T.RECORD_SOURCE_ID    = COALESCE(NULLIF(TRIM(S.RECORD_SOURCE_ID), ''),    SILVER.NVL_STR(T.RECORD_SOURCE_ID)),
  T.UPDATED_AT          = CURRENT_TIMESTAMP()

WHEN NOT MATCHED THEN INSERT (
  PATIENT_ID, FIRST_NAME, LAST_NAME, MIDDLE, PREFIX, SUFFIX, MAIDEN,
  BIRTH_DATE, DEATH_DATE, GENDER, MARITAL_STATUS, ETHNICITY, SSN,
  DRIVERS_LICENSE, PASSPORT, BIRTHPLACE, ADDRESS, CITY, STATE, ZIP, COUNTY, FIPS,
  LAT, LON, HEALTHCARE_EXPENSES, HEALTHCARE_COVERAGE, INCOME,
  SOURCE_SYSTEM, RECORD_SOURCE_ID, CREATED_AT, UPDATED_AT
) VALUES (
  S.PATIENT_ID, S.FIRST_NAME, S.LAST_NAME, S.MIDDLE, S.PREFIX, S.SUFFIX, S.MAIDEN,
  S.BIRTH_DATE, S.DEATH_DATE, S.GENDER, S.MARITAL_STATUS, S.ETHNICITY, S.SSN,
  S.DRIVERS_LICENSE, S.PASSPORT, S.BIRTHPLACE, S.ADDRESS, S.CITY, S.STATE, S.ZIP, S.COUNTY, S.FIPS,
  S.LAT, S.LON, S.HEALTHCARE_EXPENSES, S.HEALTHCARE_COVERAGE, S.INCOME,
  S.SOURCE_SYSTEM, S.RECORD_SOURCE_ID, S.CREATED_AT, S.UPDATED_AT
);



In [ ]:
CREATE OR REPLACE TABLE CSV_FINAL_ASSIGNMENT.SILVER.SILVER_ENCOUNTERS (
  encounter_id     VARCHAR,
  patient_id       VARCHAR,
  admit_ts         TIMESTAMP_NTZ,
  discharge_ts     TIMESTAMP_NTZ,
  encounter_class  VARCHAR,
  preadmit_number  VARCHAR,
  facility_id      VARCHAR,
  source_system    VARCHAR
);


-- Start with a clean load for this run
TRUNCATE TABLE CSV_FINAL_ASSIGNMENT.SILVER.SILVER_ENCOUNTERS;

-- CSV ENCOUNTERS -> SILVER

INSERT INTO CSV_FINAL_ASSIGNMENT.SILVER.SILVER_ENCOUNTERS (
  encounter_id,
  patient_id,
  admit_ts,
  discharge_ts,
  encounter_class,
  preadmit_number,
  facility_id,
  source_system
)
SELECT
  CAST(CODE AS VARCHAR)                  AS encounter_id,
  CAST(PATIENT AS VARCHAR)               AS patient_id,
  -- If these are timestamps or ISO strings, cast them; otherwise use as-is
  TO_TIMESTAMP_NTZ("START")              AS admit_ts,
  TO_TIMESTAMP_NTZ("STOP")               AS discharge_ts,
  UPPER(TRIM(ENCOUNTERCLASS))            AS encounter_class,
  NULL                                   AS preadmit_number,
  NULL                                   AS facility_id,
  'CSV'                                  AS source_system
FROM CSV_FINAL_ASSIGNMENT.CSV.ENCOUNTERS;

-- HL7 PV1 + PID -> SILVER
INSERT INTO CSV_FINAL_ASSIGNMENT.SILVER.SILVER_ENCOUNTERS (
  encounter_id,
  patient_id,
  admit_ts,         -- TIMESTAMP_NTZ in target
  discharge_ts,     -- TIMESTAMP_NTZ in target
  encounter_class,
  preadmit_number,
  facility_id,
  source_system
)
SELECT
  CAST(pv1.VISIT_NUMBER AS VARCHAR)          AS encounter_id,
  CAST(pid.PATIENT_ID_LIST AS VARCHAR)       AS patient_id,
  CAST(pv1.ADMIT_DT_TZ AS TIMESTAMP_NTZ)     AS admit_ts,        -- <-- conversion
  CAST(pv1.DISCHARGE_DT_TZ AS TIMESTAMP_NTZ) AS discharge_ts,    -- <-- conversion
  UPPER(TRIM(pv1.PATIENT_CLASS))             AS encounter_class,
  NULLIF(TRIM(pv1.PREADMIT_NUMBER), '')      AS preadmit_number,
  NULLIF(TRIM(pv1.LOC_FACILITY), '')         AS facility_id,
  'HL7'                                      AS source_system
FROM HL7_FINAL_ASSIGNMENT.HL7.HL7_BRONZE_PV1 pv1
LEFT JOIN HL7_FINAL_ASSIGNMENT.HL7.HL7_BRONZE_PID pid
       ON pv1.MESSAGE_ID = pid.MESSAGE_ID;
--select * from HL7_FINAL_ASSIGNMENT.HL7.HL7_BRONZE_PV1
-- C-CDA ENCOUNTERS -> SILVER
--SELECT DISTINCT LOC_FACILITY FROM  HL7_FINAL_ASSIGNMENT.HL7.HL7_BRONZE_PV1;
INSERT INTO CSV_FINAL_ASSIGNMENT.SILVER.SILVER_ENCOUNTERS
SELECT
  MD5(CONCAT(COALESCE(DOCUMENT_ID,''),'|',COALESCE(START_ISO,''))) AS encounter_id,
  CAST(PATIENT_ID AS VARCHAR)                                      AS patient_id,
  TO_TIMESTAMP_NTZ(START_ISO)                                      AS admit_ts,
  TO_TIMESTAMP_NTZ(STOP_ISO)                                       AS discharge_ts,
  'OUTPATIENT'                                                     AS encounter_class,  -- adjust if you can infer inpatient
  NULL                                                             AS preadmit_number,
  NULL                                                             AS facility_id,
  'CCDA'                                                           AS source_system
FROM CCDA_FINAL_ASSIGNMENT.CCDA.CCDA_ENCOUNTERS;

-- select * from CSV_FINAL_ASSIGNMENT.SILVER.SILVER_ENCOUNTERS WHERE FACILITY_ID IS NOT NULL ;
-- select COUNT(*) from CSV_FINAL_ASSIGNMENT.SILVER.SILVER_ENCOUNTERS;